In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Codegenerator

In [84]:
def a(i,j):
    return "arg.a{}{}".format(i,j)
def b(i):
    return "arg.b{}".format(i)

def gen_detA(ix, ii=None):
    parity , first = 1 , True
    for p in itertools.permutations(ix):
        if not first:
            print(" + " if parity == 1 else " - ", end="")
        else:
            first = False
        parity = -parity
        prod = (a(j,i) if i != ii else b(j) for i,j in enumerate(p))
        print("*".join(prod), end="")            

In [85]:
import itertools
def generate(d, n):
    if d > 9 or n > 9: raise(ValueError("singledigits only, sorry"))
    print("n = 0")
    for ii in range(d):
        print("v{} = 0".format(ii))
    print()

    for i, ix in enumerate(itertools.combinations(range(n), d)):
        print("d{} = ".format(i), end='')
        gen_detA(ix)        
        print()
        print("if d{} != 0:".format(i))
        for ii in range(d):
            print("    x{} = (".format(ii), end='')
            gen_detA(ix,ii)
            print(") / d{}".format(i))
        for nn in range(n):
            if nn in ix: continue
            s = ("{}*x{}".format(a(nn,ii),ii) for ii in range(d))
            print("    c{} = {} <= {}".format(nn," + ".join(s), b(nn)))
        c = ("c{}".format(nn) for nn in range(n) if nn not in ix)
        print("    if {}:".format(" and ".join(c)))  
        print("        n += 1")
        for ii in range(d):
            print("        v{} += x{}".format(ii,ii))        
        print()

    print("print(\"n =\", n)".format(ii))
    
    for ii in range(d):
        print("v{} /= n".format(ii))
    r = ",".join("v{}".format(ii) for ii in range(d))
    print("return ({})".format(r))

In [86]:
generate(2,4)

n = 0
v0 = 0
v1 = 0

d0 = arg.a00*arg.a11 - arg.a10*arg.a01
if d0 != 0:
    x0 = (arg.b0*arg.a11 - arg.b1*arg.a01) / d0
    x1 = (arg.a00*arg.b1 - arg.a10*arg.b0) / d0
    c2 = arg.a20*x0 + arg.a21*x1 <= arg.b2
    c3 = arg.a30*x0 + arg.a31*x1 <= arg.b3
    if c2 and c3:
        n += 1
        v0 += x0
        v1 += x1

d1 = arg.a00*arg.a21 - arg.a20*arg.a01
if d1 != 0:
    x0 = (arg.b0*arg.a21 - arg.b2*arg.a01) / d1
    x1 = (arg.a00*arg.b2 - arg.a20*arg.b0) / d1
    c1 = arg.a10*x0 + arg.a11*x1 <= arg.b1
    c3 = arg.a30*x0 + arg.a31*x1 <= arg.b3
    if c1 and c3:
        n += 1
        v0 += x0
        v1 += x1

d2 = arg.a00*arg.a31 - arg.a30*arg.a01
if d2 != 0:
    x0 = (arg.b0*arg.a31 - arg.b3*arg.a01) / d2
    x1 = (arg.a00*arg.b3 - arg.a30*arg.b0) / d2
    c1 = arg.a10*x0 + arg.a11*x1 <= arg.b1
    c2 = arg.a20*x0 + arg.a21*x1 <= arg.b2
    if c1 and c2:
        n += 1
        v0 += x0
        v1 += x1

d3 = arg.a10*arg.a21 - arg.a20*arg.a11
if d3 != 0:
    x0 = (arg.b1*arg.a21 

## Testing it

In [103]:
%%capture t
generate(*A.shape)

In [104]:
# This generates a "sample" function
code = ["   " + l for l in t.stdout.splitlines()]
code = ["def sample(arg):"] + code 
code = "\n".join(code)
exec(code)

def mk_sample(A, B):
    arg = type('arg', (object,), {})()
    for i, j in itertools.product(*map(range, A.shape)):
        setattr(arg, "a{}{}".format(j,i), A[i,j])
    for j in range(B.shape[0]):
        setattr(arg, "b{}".format(j), B[j])
    return sample(arg)

In [110]:
print(code)

def sample(arg):
   n = 0
   v0 = 0
   v1 = 0
   
   d0 = arg.a00*arg.a11 - arg.a10*arg.a01
   if d0 != 0:
       x0 = (arg.b0*arg.a11 - arg.b1*arg.a01) / d0
       x1 = (arg.a00*arg.b1 - arg.a10*arg.b0) / d0
       c2 = arg.a20*x0 + arg.a21*x1 <= arg.b2
       c3 = arg.a30*x0 + arg.a31*x1 <= arg.b3
       if c2 and c3:
           n += 1
           v0 += x0
           v1 += x1
   
   d1 = arg.a00*arg.a21 - arg.a20*arg.a01
   if d1 != 0:
       x0 = (arg.b0*arg.a21 - arg.b2*arg.a01) / d1
       x1 = (arg.a00*arg.b2 - arg.a20*arg.b0) / d1
       c1 = arg.a10*x0 + arg.a11*x1 <= arg.b1
       c3 = arg.a30*x0 + arg.a31*x1 <= arg.b3
       if c1 and c3:
           n += 1
           v0 += x0
           v1 += x1
   
   d2 = arg.a00*arg.a31 - arg.a30*arg.a01
   if d2 != 0:
       x0 = (arg.b0*arg.a31 - arg.b3*arg.a01) / d2
       x1 = (arg.a00*arg.b3 - arg.a30*arg.b0) / d2
       c1 = arg.a10*x0 + arg.a11*x1 <= arg.b1
       c2 = arg.a20*x0 + arg.a21*x1 <= arg.b2
       if c1 and c2:
          

In [113]:
from utils import parse_constraints
constraints = """
  x <= 3
  x >= 2
  x + y >= 0
  y + 0.5 * x <= 2 
"""
variables = ['x','y']

A, B = parse_constraints(constraints, variables)
A, B = np.array([A[v] for v in variables]), np.array(B)

print(A)
print(B)

[[ 1.  -1.  -1.   0.5]
 [ 0.  -0.  -1.   1. ]]
[ 3. -2. -0.  2.]


In [114]:
mk_sample(A,B)

n = 4


(2.5, -0.875)

## Test2

In [115]:
from utils import parse_constraints
constraints = """
  x <= 2
  x >= 0
  x + y >= 0
  y + 0.5 * x <= 2  
  z >= 2
  z + y + x <= 5
"""
variables = ['x','y','z']

A, B = parse_constraints(constraints, variables)
A, B = np.array([A[v] for v in variables]), np.array(B)

print(A)
print(B)

[[ 1.  -1.  -1.   0.5 -0.   1. ]
 [ 0.  -0.  -1.   1.  -0.   1. ]
 [ 0.  -0.  -0.   0.  -1.   1. ]]
[ 2. -0. -0.  2. -2.  5.]


In [116]:
%%capture t
generate(*A.shape)

In [117]:
# This generates a "sample" function
code = ["   " + l for l in t.stdout.splitlines()]
code = ["def sample(arg):"] + code 
code = "\n".join(code)
exec(code)

In [118]:
arg = type('arg', (object,), {})()
for i, j in itertools.product(*map(range, A.shape)):
    setattr(arg, "a{}{}".format(j,i), A[i,j])
for j in range(B.shape[0]):
    setattr(arg, "b{}".format(j), B[j])
sample(arg)

n = 7


(0.6666666666666666, 1.0952380952380951, 3.0)

In [34]:
with open("code.py", "w") as f:
    f.write(code)

In [69]:
with open("code.py", "r") as f:
    code = f.read() 
    exec(code)

In [70]:
print(code)

def sample(arg):
   n = 0
   v0 = 0
   v1 = 0
   
   d0 = arg.a00*arg.a11 - arg.a10*arg.a01
   print("d0 =",d0)
   if d0 != 0:
       x0 = (arg.b0*arg.a11 - arg.b0*arg.a01) / d0
       x1 = (arg.a00*arg.b1 - arg.a10*arg.b1) / d0
       print("  - d0",x0,x1)
       c2 = arg.a20*x0 + arg.a21*x1 <= arg.b2
       c3 = arg.a30*x0 + arg.a31*x1 <= arg.b3
       if c2 and c3:
           n += 1
           v0 += x0
           v1 += x1
   
   d1 = arg.a00*arg.a21 - arg.a20*arg.a01
   print("d1 =",d1)
   if d1 != 0:
       x0 = (arg.b0*arg.a21 - arg.b0*arg.a01) / d1
       x1 = (arg.a00*arg.b1 - arg.a20*arg.b1) / d1
       print("  - d1",x0,x1)
       c1 = arg.a10*x0 + arg.a11*x1 <= arg.b1
       c3 = arg.a30*x0 + arg.a31*x1 <= arg.b3
       if c1 and c3:
           n += 1
           v0 += x0
           v1 += x1
   
   d2 = arg.a00*arg.a31 - arg.a30*arg.a01
   print("d2 =",d2)
   if d2 != 0:
       x0 = (arg.b0*arg.a31 - arg.b0*arg.a01) / d2
       x1 = (arg.a00*arg.b1 - arg.a30*arg.b1) / d2
     